In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from IPython.core.display_functions import display
from biogeme.expressions import Expression, Beta, Variable
from biogeme.models import loglogit, nested
from biogeme.segmentation import DiscreteSegmentationTuple, segmented_beta
from biogeme.tools.likelihood_ratio import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import boxcox, loglogit, lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from pandas import Series, DataFrame


In [2]:
df = pd.read_csv("lpmc01.dat", sep = '\t')
database = db.Database('lpmc', df)
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

columns_to_remove = ['cost_driving_ccharge', 'cost_driving_fuel', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_int', 'dur_pt_bus']
df = df.drop(columns=columns_to_remove)
display(df)


,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,car_ownership,distance,dur_walking,dur_cycling,pt_interchanges,dur_driving,cost_transit,driving_traffic_percent,cost_driving,dur_pt
0,12,1,1,0,4,3,1,5,0.0,1,...,2,3379,0.833889,0.242222,0,0.141389,0.0,0.090373,0.51,0.332778
1,17,3,1,1,3,1,6,1,1.0,1,...,0,1901,0.512222,0.141667,1,0.115556,3.0,0.033654,0.33,0.423611
2,51,12,1,1,4,5,2,1,1.0,1,...,1,7835,1.823611,0.554444,1,0.355556,3.0,0.302344,1.12,0.763056
3,67,13,1,6,4,3,1,5,0.0,1,...,1,3355,0.876944,0.240833,0,0.206944,0.0,0.159732,0.67,0.424167
4,74,14,0,3,4,3,1,5,0.0,1,...,2,792,0.230000,0.067222,0,0.067778,0.0,0.151639,0.20,0.170278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81003,17593,0,1,4,5,1,1,1.0,3,...,2,3399,0.811667,0.228333,0,0.156389,1.5,0.314387,0.51,0.269167
4996,81009,17595,0,2,3,3,6,1,1.0,3,...,0,3586,0.810278,0.226667,0,0.257500,1.5,0.354908,0.57,0.370000
4997,81038,17604,1,4,4,3,5,1,1.0,3,...,2,1607,0.383889,0.120556,0,0.157778,1.5,0.366197,0.55,0.256389
4998,81071,17610,0,0,4,3,2,1,1.0,3,...,1,6003,1.514722,0.513056,0,0.259167,1.5,0.233655,0.88,0.606111


## Variable Definition

In [3]:
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
driving_traffic_percent = Variable('driving_traffic_percent')


In [4]:
all_results = {}

In [5]:
# MODEL 0
time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt = Beta(name='beta_tt', value=0, lowerbound=None, upperbound=None, status=0)

v_walking = beta_tt * time_walking
v_cycling = asc_cycling + beta_tt * time_cycling 
v_pt = asc_pt + beta_tt * time_pt + beta_cost * cost_transit
v_driving = asc_driving + beta_tt * time_driving + beta_cost * cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'

results = biogeme.estimate()
model_0_loglike = results.data.logLike
model_0_numParam = results.get_estimated_parameters().shape[0]

all_results['Model_0'] = biogeme.estimate()


In [6]:
#MODEL 1

time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)

v_walking = beta_tt_walking * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + beta_cost * cost_transit
v_driving = asc_driving + beta_tt_driving * time_driving + beta_cost * cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_1'

results = biogeme.estimate()
model_1_loglike = results.data.logLike
model_1_numParam = results.get_estimated_parameters().shape[0]

# I use MLR test, where this is the unrestricted model, and model 0 is the restricted one.
# If I can reject the null hypothesis, then this will become the preferred model.

likelihood_ratio_test([model_0_loglike, model_0_numParam], [model_1_loglike, model_1_numParam], 0.01)

# Ideally, should create 3 models: one with only alternate specific TT, one with alternate specific costs,
# and one with both. Then we should MLR test all possible combinations to have evidence to choose a 
# preferred one. 

print(results.get_estimated_parameters())


all_results['Model_1'] = biogeme.estimate()


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling     -4.602389      0.197139   -23.345935           0.0
asc_driving     -2.115335      0.144925   -14.596083           0.0
asc_pt          -2.599608      0.146555   -17.738104           0.0
beta_cost       -0.180784      0.017544   -10.304682           0.0
beta_tt_cycling -6.462341      0.483811   -13.357163           0.0
beta_tt_driving -6.623131      0.379885   -17.434547           0.0
beta_tt_pt      -3.494231      0.244071   -14.316463           0.0
beta_tt_walking -9.065158      0.456126   -19.874248           0.0


In [22]:
# Adding alternate specifc costs 

asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost_pt = Beta(name='beta_cost_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_cost_driving = Beta(name='beta_cost_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)

v_walking = beta_tt_walking * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + beta_cost_pt * cost_transit
v_driving = asc_driving + beta_tt_driving * time_driving + beta_cost_driving * cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_1_plus'

results = biogeme.estimate()
model_1_plus_loglike = results.data.logLike
model_1_plus_numParam = results.get_estimated_parameters().shape[0]


In [7]:
# Interaction Age / Travel Time

# Define driving cost
time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

mx_age = df['age'].max()

# Interacting age with ASCs and travel time

## Does it make sense to interact with asc??

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * cost_transit
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_2_tt'

results = biogeme.estimate()
model_2_tt_loglike = results.data.logLike
model_2_tt__numParam = results.get_estimated_parameters().shape[0]

print(results.get_estimated_parameters())

all_results['Model_2_tt'] = biogeme.estimate()



                             Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling              -4.618741      0.200583   -23.026576  0.000000e+00
asc_driving              -2.156608      0.150105   -14.367306  0.000000e+00
asc_pt                   -2.612314      0.149861   -17.431624  0.000000e+00
beta_cost                -0.182681      0.017772   -10.278879  0.000000e+00
beta_tt_cycling          -4.682452      0.865111    -5.412543  6.213609e-08
beta_tt_cycling_interact -4.489886      1.941357    -2.312756  2.073604e-02
beta_tt_driving          -5.824642      0.882429    -6.600696  4.092326e-11
beta_tt_driving_interact -1.944159      2.054091    -0.946481  3.439032e-01
beta_tt_pt               -2.274892      0.486566    -4.675398  2.933843e-06
beta_tt_pt_interact      -3.055052      1.128707    -2.706683  6.795919e-03
beta_tt_walking          -7.455478      0.504921   -14.765621  0.000000e+00
beta_tt_walking_interact -4.133963      1.082997    -3.817151  1.350013e-04


In [8]:
# Interaction Age / ASCs

time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_pt_Age = Beta(name='asc_pt_Age', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling_Age = Beta(name='asc_cycling_Age', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving_Age = Beta(name='asc_driving_Age', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)

mx_age = df['age'].max()

# Interacting age with ASCs and travel time

## Does it make sense to interact with asc??

v_walking = beta_tt_walking * time_walking
v_cycling = asc_cycling + (1/mx_age) * age * asc_cycling_Age  + beta_tt_cycling * time_cycling 
v_pt = asc_pt + (1/mx_age) * age * asc_pt_Age  + beta_tt_pt * time_pt + beta_cost * cost_transit
v_driving = asc_driving + (1/mx_age) * age * asc_driving_Age  + beta_tt_driving * time_driving + beta_cost * cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_2_asc'

results = biogeme.estimate()
model_2_asc_loglike = results.data.logLike
model_2_asc_numParam = results.get_estimated_parameters().shape[0]

all_results['Model_2_Asc'] = biogeme.estimate()


In [9]:
# MODEL 3


time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

ell_cost = Beta('lambda_cost', 1, -10, 10, 0)
boxcox_cost_pt = boxcox(cost_transit, ell_cost)
boxcox_cost_driving = boxcox(cost_driving, ell_cost)

mx_age = df['age'].max()

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * boxcox_cost_pt
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * boxcox_cost_driving

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3_boxcox'

results = biogeme.estimate()
model_3_boxcox_loglike = results.data.logLike
model_3_boxcox_numParam = results.get_estimated_parameters().shape[0]

all_results['Model_3_boxcox'] = biogeme.estimate()



In [10]:
# MODEL 3

time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )


beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

cost_drive_power = power_series(cost_driving)
cost_pt_power = power_series(cost_transit)

mx_age = df['age'].max()

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * cost_pt_power
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * cost_drive_power

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3_power_series'

results = biogeme.estimate()
model_3_power_Series_loglike = results.data.logLike
model_3_power_Series_numParam = results.get_estimated_parameters().shape[0]

all_results['Model_3_power_series'] = biogeme.estimate()



In [11]:
# MODEL 3

time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

cost_drive_power = power_series(cost_driving)
cost_pt_power = power_series(cost_transit)

mx_age = df['age'].max()

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * cost_pt_power
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * cost_drive_power

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3_power_series'

results = biogeme.estimate()
model_3_power_Series_loglike = results.data.logLike
model_3_power_Series_numParam = results.get_estimated_parameters().shape[0]

all_results['Model_3_power_series'] = biogeme.estimate()



In [24]:
# MODEL 3


time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

cost_drive_power = power_series(cost_driving)
cost_pt_power = power_series(cost_transit)

mx_age = df['age'].max()

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * cost_pt_power
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * cost_drive_power

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

logprob = loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3'

model_3_results = biogeme.estimate()
model_3_loglike = model_3_results.data.logLike
model_3_numParam = model_3_results.get_estimated_parameters().shape[0]

all_results['Model_3'] = biogeme.estimate()


In [14]:
# MODEL 4


# Define driving cost


time_pt = dur_pt
time_cycling = dur_cycling
time_walking = dur_walking  
time_driving = dur_driving

# Model normalized with asc_walking = 0
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )

beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)
beta_tt_driving_interact = Beta(name='beta_tt_driving_interact', value=0, lowerbound=None, upperbound=None, status=0)

cost_drive_power = power_series(cost_driving)
cost_pt_power = power_series(cost_transit)

mx_age = df['age'].max()

v_walking = beta_tt_walking * time_walking +  (1/mx_age) * beta_tt_walking_interact * age * time_walking
v_cycling = asc_cycling + beta_tt_cycling * time_cycling + (1/mx_age) * beta_tt_cycling_interact * age * time_cycling 
v_pt = asc_pt + beta_tt_pt * time_pt + (1/mx_age) * beta_tt_pt_interact * age * time_pt + beta_cost * cost_pt_power
v_driving = asc_driving + beta_tt_driving * time_driving + (1/mx_age) * beta_tt_driving_interact * age * time_driving + beta_cost * cost_drive_power

V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)
nest_a = OneNestForNestedLogit(nest_param=mu_a, list_of_alternatives=[1, 2], name='slow modes')
nest_b = OneNestForNestedLogit(nest_param=mu_b, list_of_alternatives=[3, 4], name='faster modes')
nests = NestsForNestedLogit(choice_set=list(V), tuple_of_nests=(nest_a, nest_b))

logprob_m4 = lognested(V, None, nests, travel_mode)

model_4 = bio.BIOGEME(database, logprob_m4)
model_4.modelName = 'model_4'

results_m4 = model_4.estimate()
model_4_loglike = results_m4.data.logLike
model_4__numParam = results_m4.get_estimated_parameters().shape[0]

results_m4.get_estimated_parameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycling,-6.513651,0.309194,-21.066565,0.000000e+00
asc_driving,-2.690262,0.181420,-14.828904,0.000000e+00
asc_pt,-3.535248,0.271539,-13.019281,0.000000e+00
beta_cost,-0.000007,0.000001,-5.251045,1.512386e-07
beta_tt_cycling,-3.310087,1.014925,-3.261412,1.108590e-03
beta_tt_cycling_interact,-6.056864,2.364816,-2.561241,1.042991e-02
beta_tt_driving,-8.042007,1.565272,-5.137771,2.780164e-07
beta_tt_driving_interact,-4.840528,2.849909,-1.698485,8.941617e-02
beta_tt_pt,-3.059765,0.775528,-3.945398,7.966754e-05
beta_tt_pt_interact,-5.610293,1.728685,-3.245410,1.172815e-03


In [13]:
comparison_table, _ = compile_estimation_results(all_results)
display(comparison_table)





,Model_0,Model_1,Model_2_tt,Model_2_Asc,Model_3_boxcox,Model_3_power_series,Model_3
Number of estimated parameters,5,8,12,11,13,14,14
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4581.895656,-4223.98596,-4193.755258,-4202.066678,-4194.800317,-4185.483739,-4185.483777
Akaike Information Criterion,9173.791312,8463.97192,8411.510515,8426.133356,8415.600634,8398.967477,8398.967554
Bayesian Information Criterion,9206.377278,8516.109465,8489.716833,8497.822481,8500.324145,8490.208182,8490.208259
asc_cycling (t-test),-3.78 (-36.6),-4.6 (-23.3),-4.62 (-23),-4.81 (-18.1),-4.75 (-21.9),-4.62 (-22.9),-4.62 (-22.9)
asc_driving (t-test),-1.29 (-16.2),-2.12 (-14.6),-2.16 (-14.4),-2.76 (-14.8),-2.89 (-13.6),-2.14 (-14.2),-2.14 (-14.2)
asc_pt (t-test),-0.526 (-9.6),-2.6 (-17.7),-2.61 (-17.4),-3.01 (-16.1),-2.79 (-17.4),-2.53 (-16.4),-2.54 (-16.4)
beta_cost (t-test),-0.191 (-13),-0.181 (-10.3),-0.183 (-10.3),-0.182 (-10.3),-0.468 (-8.91),-0.3 (-4.6),-0.3 (-4.62)
beta_tt (t-test),-5.6 (-27.5),,,,,,


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycling,-6.358386,0.297406,-21.379455,0.000000e+00
asc_driving,-1.875674,0.197641,-9.490307,0.000000e+00
asc_pt,-3.305568,0.234671,-14.085976,0.000000e+00
beta_cost,0.002647,0.000397,6.673410,2.499267e-11
beta_tt_cycling,-3.841209,0.999962,-3.841356,1.223563e-04
beta_tt_cycling_interact,-4.960457,2.173664,-2.282071,2.248512e-02
beta_tt_driving,-7.223613,1.373639,-5.258740,1.450459e-07
beta_tt_driving_interact,-3.747157,2.536040,-1.477562,1.395250e-01
beta_tt_pt,-3.056179,0.702483,-4.350538,1.358036e-05
beta_tt_pt_interact,-4.574871,1.511788,-3.026132,2.477038e-03


In [47]:
data_filtered = df

populations = {
    'female_45_less': 2841376,
    'female_45_or_more': 1519948,
    'male_45_less': 2929408,
    'male_45_or_more': 1379198,
}

total_pop = sum(populations.values())

filters = {
    'male_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 0),
    'male_45_less': (data_filtered.age < 45) & (data_filtered.female == 0),
    'female_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 1),
    'female_45_less': (data_filtered.age < 45) & (data_filtered.female == 1),
}

sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(sample_segments)

total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

weights = {
    segment_name: populations[segment_name] * total_sample / (segment_size * total_pop)
    for segment_name, segment_size in sample_segments.items()
}
print(weights)

{'male_45_or_more': np.int64(896), 'male_45_less': np.int64(1442), 'female_45_or_more': np.int64(984), 'female_45_less': np.int64(1678)}
Sample size: 5000
{'male_45_or_more': np.float64(0.8877139043468962), 'male_45_less': np.float64(1.1715720875375348), 'female_45_or_more': np.float64(0.890816074423909), 'female_45_less': np.float64(0.9765425353056789)}


In [ ]:
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp

from biogeme import models

for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]

prob_pt =  models.logit(V, None, 3)  # Probability of public transport (3 is the index for PT in `V`)
prob_car = models.logit(V, None, 4)  # Probability of driving (4 is the index for driving in `V`)
prob_walk = models.logit(V, None, 1)  # Probability of walking (1 is the index for walking in `V`)
prob_cycle = models.logit(V, None, 2)  # Probability of cycling (2 is the index for cycling in `V`)

weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
}

data_filtered["cost_driving"] += 1.5
database = db.Database('london', data_filtered)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(model_3_results.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)
simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.887714,0.427566,0.537432,2.701982e-03,0.032300
1,0.976543,0.164111,0.550406,2.053342e-01,0.080148
2,0.890816,0.266528,0.708024,2.082216e-06,0.025446
3,0.887714,0.474141,0.466367,4.336483e-03,0.055156
4,0.890816,0.216154,0.215861,5.343256e-01,0.033659
...,...,...,...,...,...
4995,0.887714,0.456929,0.487574,8.776025e-03,0.046721
4996,1.171572,0.545175,0.342124,2.956528e-02,0.083136
4997,0.976543,0.282501,0.262336,3.949157e-01,0.060246
4998,0.976543,0.408293,0.570741,7.212419e-05,0.020894


In [31]:
market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')


Market share for pt: 42.9%
Market share for car: 32.5%
Market share for walk: 20.4%
Market share for cycling: 4.2%


In [59]:
model_forecast.bootstrap_samples = 100
results_bootstrapping = model_forecast.estimate(run_bootstrap=True)

betas = model_forecast.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)
    
display(left)

display(right)

AttributeError: 'bioResults' object has no attribute 'estimate'

In [58]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

NameError: name 'left' is not defined

In [33]:
# Example mapping
labels = {1: 'walk', 2: 'cycling', 3: 'pt', 4: 'car'}

# Map the travel_mode column to the labels
data_filtered['mode_label'] = data_filtered['travel_mode'].map(labels)

# Calculate market shares
market_shares = (
    data_filtered['mode_label']
    .value_counts(normalize=True)  # Get proportions
    .sort_index()  # Ensure consistent order
    * 100  # Convert to percentage
)

# Print market shares
for mode, share in market_shares.items():
    print(f"Market share for {mode}: {share:.1f}%")

Market share for car: 44.0%
Market share for cycling: 3.3%
Market share for pt: 35.3%
Market share for walk: 17.4%


In [34]:
print(data_filtered["mode_label"].dtype)
print(data_filtered["mode_label"].unique())

object
['car' 'pt' 'walk' 'cycling']


In [35]:
from biogeme.expressions import Derive
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models

for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]

weight = Variable('weight')
vot_pt = Derive(v_pt, 'dur_pt') / Derive(v_pt, 'cost_transit')
vot_car = Derive(v_driving, 'dur_driving') / Derive(v_driving, 'cost_driving')
simulate = {
    'weight': weight,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
}
biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(model_3_results.get_beta_values())

print(-simulated_values['WTP PT time'].mean())
print(-simulated_values['WTP CAR time'].mean())

Parameter asc_cycling not present in the model.
Parameter beta_tt_cycling not present in the model.
Parameter beta_tt_cycling_interact not present in the model.
Parameter beta_tt_walking not present in the model.
Parameter beta_tt_walking_interact not present in the model.


-13.22320906461031
-66.91744913378146


In [ ]:
prob_pt =  models.logit(V, None, 3)  # Probability of public transport (3 is the index for PT in `V`)
prob_car = models.logit(V, None, 4)  # Probability of driving (4 is the index for driving in `V`)
prob_walk = models.logit(V, None, 1)  # Probability of walking (1 is the index for walking in `V`)
prob_cycle = models.logit(V, None, 2)  # Probability of cycling (2 is the index for cycling in `V`)

direct_elas_car_cost = Derive(prob_car, 'cost_driving') * dur_driving / prob_car
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt

cross_elas_prob_car_pt_cost = Derive(prob_car, 'cost_transit') * cost_transit / prob_car
cross_elas_prob_walk_pt_cost = Derive(prob_walk, 'cost_transit') * cost_transit / prob_walk
cross_elas_prob_cycle_pt_cost = Derive(prob_cycle, 'cost_transit') * cost_transit / prob_cycle

cross_elas_prob_pt_car_cost = Derive(prob_cycle, 'cost_driving') * cost_driving / prob_cycle


simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
    'direct_elas_pt_cost': direct_elas_pt_cost,
    'direct_elas_car_cost': direct_elas_car_cost,
    'cross_elas_prob_car_pt_cost': cross_elas_prob_car_pt_cost,
    'cross_elas_prob_walk_pt_cost': cross_elas_prob_walk_pt_cost,
    'cross_elas_prob_cycle_pt_cost': cross_elas_prob_cycle_pt_cost
}

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(model_3_results.get_beta_values())

simulated_values['numerator_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. pt']
    * simulated_values['direct_elas_pt_cost']
)

simulated_values['denominator_pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

agg_elast_pt_cost = (
    simulated_values['numerator_pt_cost'].sum()
    / simulated_values['denominator_pt'].sum()
)

print(f'Aggregate PT elasticity wrt cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['direct_elas_car_cost']
)

simulated_values['denominator_car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

agg_elast_car_cost = (
    simulated_values['numerator_car_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Aggregate CAR elasticity wrt cost: {agg_elast_car_cost:.3g}')

simulated_values['numerator_cross_car_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['cross_elas_prob_car_pt_cost']
)

agg_elast_pt_cost = (
    simulated_values['numerator_cross_car_pt_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Cross CAR elasticity wrt PT cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_cross_walk_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_pt_cost']
)

simulated_values['denominator_walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)

agg_elast_cross_walk_pt_time = (
    simulated_values['numerator_cross_walk_pt_time'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt PT time: {agg_elast_cross_walk_pt_time:.3g}')

simulated_values['numerator_cross_cycle_pt_time'] = (
    simulated_values['weight']
    * simulated_values['Prob. cycle']
    * simulated_values['cross_elas_prob_cycle_pt_time']
)

simulated_values['denominator_cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

agg_elast_cross_cycle_pt_time = (
    simulated_values['numerator_cross_cycle_pt_time'].sum()
    / simulated_values['denominator_cycle'].sum()
)

print(f'Cross CYCLE elasticity wrt PT time: {agg_elast_cross_cycle_pt_time:.3g}')


# NOW FOR CAR TIME




Aggregate PT elasticity wrt time: -0.704
Aggregate CAR elasticity wrt time: -0.965
Cross CAR elasticity wrt PT time: 0.736
Cross WALK elasticity wrt PT time: 0.164
Cross CYCLE elasticity wrt PT time: 0.695
